In [42]:
import pandas as pd
import geopandas as gpd
import numpy as np
from sklearn.preprocessing import Normalizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC
from sklearn.svm import SVC

In [3]:
df = pd.read_csv('resultados/modelo.csv.zip')
df.fillna(0, inplace=True)
rmsp = (df.rm == 20)
subnormal = (df.tipo_do_setor == 1)
df.loc[:, 'subnormal'] = 0
df.loc[subnormal, 'subnormal'] = 1

/tmp/ipykernel_11057/1804806534.py:1: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('resultados/modelo.csv.zip')


In [4]:
dimensoes = [
    'porcentagem_domicilios_sem_coleta_de_lixo',
    'porcentagem_domicilios_sem_ligacao_rede_de_agua',
    'porcentagem_domicilios_sem_banheiros',
    'porcentagem_domicilios_sem_esgoto_ou_fossa',
    # # 'porcentagem_domicilios_tipo_comodo',
    'porcentagem_domicilios_cedidos_outra_forma',
    'porcentagem_domicilios_em_outra_condicao_de_ocupacao',
    'numero_de_banheiros_por_habitante',
    'porcentagem_responsaveis_nao_alfabetizado',
    'porcentagem_responsaveis_nao_alfabetizado_com_menos_de_30_anos',
    'porcentagem_responsaveis_renda_ate_3_salarios',
    # # 'porcentagem_responsaveis_menos_8_anos_de_estudo',
    # # 'anos_medios_estudo_responsavel',
    'renda_media_responsavel',
    'numero_de_domicilios_particulares_permanentes',
    'numero_de_domicilios_improvisados',
    'numero_de_pessoas_residentes',
    'porcentagem_de_responsaveis_com_menos_30_anos',
    'numero_medio_pessoas'
]

In [5]:
df.loc[rmsp].subnormal.value_counts()

subnormal
0    26709
1     3234
Name: count, dtype: int64

In [11]:
samples = df.loc[rmsp & (df.loc[rmsp].subnormal == 1)].shape[0]

In [12]:
x = df.loc[(rmsp) & (subnormal)]
x_ = df.loc[(rmsp) & (~subnormal)].sample(n=samples, random_state=51)

In [13]:
pd.concat([x, x_]).shape

(6468, 26)

In [14]:
X = pd.concat([x, x_]).loc[(rmsp), dimensoes]
y = pd.concat([x, x_]).loc[(rmsp), 'subnormal']

In [15]:
y.value_counts()

subnormal
1    3234
0    3234
Name: count, dtype: int64

In [36]:
random_forest_clf = RandomForestClassifier(max_depth=10, random_state=0)
random_forest_clf.fit(X,y)
random_forest_clf.score(X,y)

0.9505256648113791

In [37]:
np.unique(random_forest_clf.predict(df.loc[(rmsp), dimensoes]), return_counts=True)

(array([0, 1]), array([22976,  6967]))

In [45]:
linear_svc_clf = LinearSVC(random_state=0, tol=1e-5)
# linear_svc_clf = SVC(gamma='auto')
linear_svc_clf.fit(Normalizer().fit_transform(X),y)
linear_svc_clf.score(Normalizer().fit_transform(X),y)

0.7156771799628943

In [46]:
np.unique(linear_svc_clf.predict(Normalizer().fit_transform(df.loc[(rmsp), dimensoes])), return_counts=True)

(array([0, 1]), array([20522,  9421]))